In [1]:
import csv
import numpy as np

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision
from torchvision import transforms

from PIL import Image

import random

from torch.optim.lr_scheduler import StepLR

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

torch.cuda.empty_cache()

cuda


In [2]:
def read_csv(train_test):
  lines = []
  with open(f"lens/CogentAnnotation{train_test}.csv", "r") as file:
      
    csvreader = csv.reader(file)

    for row in csvreader:
        lines.append(np.asarray(row))

  lines.pop(0) #Removing the header of the file
  lines = np.array(lines).T #Transposing to recover the data easier

  return lines

def label_classifier(lens_labels, gender_labels, black_withe_labels):
    lens_labels_types = {"Colored": 0, "Normal": 1, "Transparent": 2}
    gender_types = {"Male": 0, "Female": 1}
    black_withe_types = {"No": 0, "Yes": 1}

    num_labels = len(lens_labels)
    for i in range(num_labels):
        lens_labels[i] = lens_labels_types[lens_labels[i]]
        gender_labels[i] = gender_types[gender_labels[i]]
        black_withe_labels[i] = black_withe_types[black_withe_labels[i]]

    lens_labels = lens_labels.astype(np.int32)
    gender_labels = gender_labels.astype(np.int8)
    black_withe_labels = black_withe_labels.astype(np.int8)

    return lens_labels, gender_labels, black_withe_labels

def normalize_array(arr):
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    return (arr - mean[:, None, None]) / std[:, None, None]

def read_images(images_path):
    images = []
    for img_name in images_path:
        img = Image.open("lens/" + img_name + ".bmp")
        img = img.resize((300,300))
        img = np.asarray(img).astype(np.float64) / 255.0
        img = np.stack((img,)*3, axis=0)
        images.append(normalize_array(img))

    return np.array(images)


def read_data(train_test, number_of_samples):

    #Picking random samples from the dataset
    number_of_samples = 1750 if number_of_samples > 1750 else number_of_samples
    samples_index = (random.sample(range(0, 1749 + 1), number_of_samples))

    #Reading from the csv all the information needed
    csv_info = read_csv(train_test)
    images_path = np.array([])
    lens_labels = np.array([])
    gender_labels = np.array([])
    black_withe_labels = np.array([])
    pupilX = np.array([])
    pupilY = np.array([])
    pupilR = np.array([])
    irisX = np.array([])
    irisY = np.array([])
    irisR = np.array([])

    for index in samples_index:
        images_path = np.append(images_path, csv_info[0][index])
        lens_labels = np.append(lens_labels, csv_info[1][index])
        gender_labels = np.append(gender_labels, csv_info[4][index])
        black_withe_labels = np.append(black_withe_labels, csv_info[5][index])
        pupilX = np.append(pupilX, csv_info[6][index])
        pupilY = np.append(pupilY, csv_info[7][index])
        pupilR = np.append(pupilR, csv_info[8][index])
        irisX = np.append(irisX, csv_info[9][index])
        irisY = np.append(irisY, csv_info[10][index])
        irisR = np.append(irisR, csv_info[11][index])


    pupilX = pupilX.astype(np.float32)
    pupilY = pupilY.astype(np.float32)
    irisX = irisX.astype(np.float32)
    irisY = irisY.astype(np.float32)
    pupilR = pupilR.astype(np.float32)
    irisR = irisR.astype(np.float32)

    #Processing the labels and reading the images
    lens_labels, gender_labels, black_withe_labels = label_classifier(lens_labels, gender_labels, black_withe_labels)
    images = read_images(images_path)

    return images, lens_labels, gender_labels, black_withe_labels, pupilX, pupilY, pupilR, irisX, irisY, irisR

images_train, lens_labels_train, gender_labels_train, black_withe_labels_train, pupilX_labels_train, pupilY_labels_train, pupilR_labels_train, irisX_labels_train, irisY_labels_train, irisR_labels_train = read_data("train", 400)
images_test, lens_labels_test, gender_labels_test, black_withe_labels_test, pupilX_labels_test, pupilY_labels_test, pupilR_labels_test, irisX_labels_test, irisY_labels_test, irisR_labels_test = read_data("test", 200)

In [3]:
#converting data to tensor
train_x = torch.from_numpy(images_train).to(torch.float32)
train_lens_label = torch.from_numpy(lens_labels_train).to(torch.float32)
train_gender_label = torch.from_numpy(gender_labels_train).to(torch.float32)
train_black_white_label = torch.from_numpy(black_withe_labels_train).to(torch.float32)
train_pupilX_label = torch.from_numpy(pupilX_labels_train).to(torch.float32)
train_pupilY_label = torch.from_numpy(pupilY_labels_train).to(torch.float32)
train_irisX__label = torch.from_numpy(irisX_labels_train).to(torch.float32)
train_irisY__label = torch.from_numpy(irisY_labels_train).to(torch.float32)
train_pupilR_label = torch.from_numpy(pupilR_labels_train).to(torch.float32)
train_irisR__label = torch.from_numpy(irisR_labels_train).to(torch.float32)



test_x = torch.from_numpy(images_test).to(torch.float32)
test_lens_label = torch.from_numpy(lens_labels_test).to(torch.float32)
test_gender_label = torch.from_numpy(gender_labels_test).to(torch.float32)
test_black_white_label = torch.from_numpy(black_withe_labels_test).to(torch.float32)
test_pupilX_label = torch.from_numpy(pupilX_labels_test).to(torch.float32)
test_pupilY_label = torch.from_numpy(pupilY_labels_test).to(torch.float32)
test_irisX__label = torch.from_numpy(irisX_labels_test).to(torch.float32)
test_irisY__label = torch.from_numpy(irisY_labels_test).to(torch.float32)
test_pupilR_label = torch.from_numpy(pupilR_labels_test).to(torch.float32)
test_irisR__label = torch.from_numpy(irisR_labels_test).to(torch.float32)

In [4]:
train = TensorDataset(train_x, train_lens_label, train_gender_label, train_black_white_label, train_pupilX_label, train_pupilY_label, train_pupilR_label, train_irisX__label, train_irisY__label, train_irisR__label)
test = TensorDataset(test_x, test_lens_label, test_gender_label, test_black_white_label, test_pupilX_label, test_pupilY_label, test_pupilR_label, test_irisX__label, test_irisY__label, test_irisR__label)

train_loaded = DataLoader(train, batch_size=8, shuffle=True)
test_loaded = DataLoader(test, batch_size=8, shuffle=True)

In [5]:
torch.cuda.empty_cache()

import torch
import torch.nn as nn
import torch.nn.functional as F

class Swish(nn.Module):
    def __init__(self, beta=1.0):
        super(Swish, self).__init__()
        self.beta = beta

    def forward(self, x):
        return x * torch.sigmoid(self.beta * x)

class lensModel(nn.Module):
    def __init__(self):
        super(lensModel, self).__init__()

        self.weights = torchvision.models.EfficientNet_B3_Weights.DEFAULT
        self.efficientNetB3 = torchvision.models.efficientnet_b3(weights= self.weights)
        self.efficientNetB3._avg_pooling = nn.Identity()
        
        self.shared_layers = nn.Sequential(
            nn.BatchNorm1d(1000),
            nn.Dropout(p=0.15),
            nn.Linear(1000, 1536),
            Swish(),

            nn.Dropout(p=0.2),
            nn.Linear(1536, 512),


            nn.Dropout(p=0.15),
            nn.Linear(512, 512),
            nn.ReLU(),
        )

        self.lens_layer = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),

            nn.BatchNorm1d(256),
            nn.Dropout(p=0.17),
            nn.Linear(256, 3),
            nn.Softmax(dim = 1)
        )

        self.gender_layer = nn.Sequential(
            nn.Linear(512, 256),
            Swish(),
            nn.Linear(256, 1),  
            nn.Sigmoid()  
        )

        self.black_white_layer = nn.Sequential(
            nn.Linear(512, 256),
            Swish(),

            nn.Linear(256, 1),  
            nn.Sigmoid()
        )

        self.pupilX_layer = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),
            Swish(),

            nn.Dropout(p=0.2),
            nn.Linear(256, 128),
            Swish(),
            
            nn.Linear(128, 1),
        )

        self.pupilY_layer = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),
            Swish(),

            nn.Dropout(p=0.2),
            nn.Linear(256, 128),
            Swish(),
            
            nn.Linear(128, 1),
        )

        self.pupilR_layer = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

        self.irisX_layer = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),
            Swish(),

            nn.Dropout(p=0.2),
            nn.Linear(256, 128),
            Swish(),
            
            nn.Linear(128, 1),
        )

        self.irisY_layer = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),
            Swish(),

            nn.Dropout(p=0.2),
            nn.Linear(256, 128),
            Swish(),
            
            nn.Linear(128, 1),
        )

        self.irisR_layer = nn.Sequential(
            nn.Dropout(p=0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        x = self.efficientNetB3(x)
        
        x = x.view(x.size(0), -1)
        
        # Check if the batch size is 1, if so, bypass the BatchNormalization layer
        if x.size(0) == 1:
            x = self.shared_layers[1:](x)  # Skips the BatchNormalization layer
        else:
            x = self.shared_layers(x)

        lens = self.lens_layer(x)
        gender = self.gender_layer(x)
        black_white = self.black_white_layer(x)
        pupilX = self.pupilX_layer(x)
        pupilY = self.pupilY_layer(x)
        pupilR = self.pupilR_layer(x)
        irisX = self.irisX_layer(x)
        irisY = self.irisY_layer(x)
        irisR = self.irisR_layer(x)
        
        return lens, gender, black_white, pupilX, pupilY, pupilR, irisX, irisY, irisR
    
model = lensModel()
model.to(device)

optimizer_lens  = torch.optim.SGD(model.lens_layer.parameters(), lr=0.0018)
optimizer_gender  = torch.optim.Adam(model.gender_layer.parameters(), lr= 0.01)
optimizer_black_white = torch.optim.Adam(model.black_white_layer.parameters(), lr= 0.01)
optimizer_pupilX = torch.optim.Adam(model.pupilX_layer.parameters(), lr=0.001)
optimizer_pupilY = torch.optim.Adam(model.pupilY_layer.parameters(), lr=0.001)
optimizer_pupilR = torch.optim.Adam(model.pupilR_layer.parameters(), lr=0.001)
optimizer_irisX = torch.optim.Adam(model.irisX_layer.parameters(), lr=0.001)
optimizer_irisY = torch.optim.Adam(model.irisY_layer.parameters(), lr=0.001)
optimizer_irisR = torch.optim.Adam(model.irisR_layer.parameters(), lr=0.001)


criterion_lens = nn.CrossEntropyLoss()
criterion_gender = nn.BCELoss()
criterion_black_white = nn.BCELoss()
criterion_pupilX = nn.BCELoss()
criterion_pupilY = nn.BCELoss()
criterion_pupilR = nn.BCELoss()
criterion_irisX = nn.BCELoss()
criterion_irisY = nn.BCELoss()
criterion_irisR = nn.BCELoss()

In [7]:
def getOutputs(model, inputs):
    lens_outputs, gender_outputs, black_white_outputs, pupilX_outputs, pupilY_outputs, pupilR_outputs, irisX_outputs, irisY_outputs, irisR_outputs = model(inputs)
    _, lens_outputs = torch.max(lens_outputs.data, 1)
    lens_outputs = lens_outputs.to(torch.float32)

    gender_outputs = gender_outputs.view(-1)  # transforming from shape [batch_size, 1] to [batch_size]
    black_white_outputs = black_white_outputs.view(-1)  # transforming from shape [batch_size, 1] to [batch_size]

    _, pupilX_outputs = torch.max(pupilX_outputs.data, 1)
    pupilX_outputs = pupilX_outputs.to(torch.float32)

    _, pupilY_outputs = torch.max(pupilY_outputs.data, 1)
    pupilY_outputs = pupilY_outputs.to(torch.float32)

    _, irisX_outputs = torch.max(irisX_outputs.data, 1)
    irisX_outputs = irisX_outputs.to(torch.float32)

    _, irisY_outputs = torch.max(irisY_outputs.data, 1)
    irisY_outputs = irisY_outputs.to(torch.float32)

    _, pupilR_outputs = torch.max(pupilR_outputs.data, 1)
    pupilR_outputs = pupilR_outputs.to(torch.float32)

    _, irisR_outputs = torch.max(irisR_outputs.data, 1)
    irisR_outputs = irisR_outputs.to(torch.float32)

    return lens_outputs, gender_outputs, black_white_outputs, pupilX_outputs, pupilY_outputs, pupilR_outputs, irisX_outputs, irisY_outputs, irisR_outputs


num_epochs = 20

model.train()
model.to(device)

for epoch in range(num_epochs):
    total_loss_lens = total_loss_gender = total_loss_black_white = total_loss_coordinates = total_loss_radius = total_loss_pupilX = total_loss_pupilY = total_loss_irisX = total_loss_irisY = total_loss_pupilR = total_loss_irisR = 0.0
    correct_lens = correct_gender = correct_black_white = correct_coordinates = correct_radius = correct_pupilX = correct_pupilY = correct_irisX = correct_irisY = correct_pupilR = correct_irisR = 0

    for batch_idx, (images, lens_label, gender_label, black_white_label, pupilX_label, pupilY_label, pupilR_label, irisX_label, irisY_label, irisR_label) in enumerate(train_loaded):
        #loading data to device
        images = images.to(device)
        lens_label = lens_label.to(device)
        gender_label = gender_label.to(device)
        black_white_label = black_white_label.to(device)
        pupilX_label = pupilX_label.to(device)
        pupilY_label = pupilY_label.to(device)
        irisX_label = irisX_label.to(device)
        irisY_label = irisY_label.to(device)
        pupilR_label = pupilR_label.to(device)
        irisR_label = irisR_label.to(device)


        optimizer_lens.zero_grad()
        optimizer_gender.zero_grad()
        optimizer_black_white.zero_grad()
        optimizer_pupilX.zero_grad()
        optimizer_pupilY.zero_grad()
        optimizer_irisX.zero_grad()
        optimizer_irisY.zero_grad()
        optimizer_pupilR.zero_grad()
        optimizer_irisR.zero_grad()

        #Receiving the model output
        lens_output, gender_output, black_white_output, pupilX_output, pupilY_output, pupilR_outputs, irisX_output, irisY_output, irisR_output = getOutputs(model=model, inputs=images)

        #Calculating the loss
        loss_lens = criterion_lens(lens_output, lens_label.float())
        loss_gender = criterion_gender(gender_output, gender_label)
        loss_black_white = criterion_black_white(black_white_output, black_white_label)

        loss_pupilX = criterion_pupilX(pupilX_output, pupilX_label)
        loss_pupilY = criterion_pupilY(pupilY_output, pupilY_label)
        loss_irisX = criterion_irisX(irisX_output, irisX_label)
        loss_irisY = criterion_irisY(irisY_output, irisY_label)
        loss_coordinates = loss_pupilX + loss_pupilY + loss_irisX + loss_irisY

        loss_pupilR = criterion_pupilR(pupilR_outputs, pupilR_label)
        loss_irisR = criterion_irisR(irisR_output, irisR_label)
        loss_radius = loss_pupilR + loss_irisR

        total_loss_lens += loss_lens.item()
        total_loss_gender += loss_gender.item()
        total_loss_black_white += loss_black_white.item()
        total_loss_coordinates += loss_coordinates.item()
        total_loss_radius += loss_radius.item()

        total_loss_pupilX += loss_pupilX.item()
        total_loss_pupilY += loss_pupilY.item()
        total_loss_irisX += loss_irisX.item()
        total_loss_irisY += loss_irisY.item()
        total_loss_pupilR += loss_pupilR.item()
        total_loss_irisR += loss_irisR.item()

        loss = loss_lens + loss_gender + loss_black_white + loss_coordinates + loss_radius

        #backprop
        loss.backward()

        #optimizing
        optimizer_lens.step()
        optimizer_gender.step()
        optimizer_black_white.step()
        optimizer_pupilX.step()
        optimizer_pupilY.step()
        optimizer_irisX.step()
        optimizer_irisY.step()
        optimizer_pupilR.step()
        optimizer_irisR.step()

        batches_done = epoch * len(train_loaded) + batch_idx
        batches_left = num_epochs * len(train_loaded) - batches_done

        if (batch_idx + 1) % 20 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}] | Batch [{batch_idx + 1}/{len(train_loaded)}] | \n\t"
                    f"Loss Lens: {total_loss_lens / (batch_idx + 1):.2f} | "
                    f"Loss Gender: {total_loss_gender / (batch_idx + 1):.2f} | "
                    f"Loss B/W: {total_loss_black_white / (batch_idx + 1):.2f} | "
                    f"loss_pupilX: {total_loss_pupilX/(batch_idx+1):.2f} | "
                    f"loss_pupilY: {total_loss_pupilY/(batch_idx+1):.2f} | "
                    f"loss_irisX: {total_loss_irisX/(batch_idx+1):.2f} | "
                    f"loss_irisY: {total_loss_irisY/(batch_idx+1):.2f} | "
                    f"loss_pupilR: {total_loss_pupilR/(batch_idx+1):.2f} | "
                    f"loss_irisR: {total_loss_irisR/(batch_idx+1):.2f}")

        #Validating the data
        correct_lens += torch.sum(lens_output == lens_label).item()
        correct_gender += torch.sum(gender_output == gender_label).item()
        correct_black_white += torch.sum(black_white_output == black_white_label).item()
        correct_pupilX += torch.sum(pupilX_output == pupilX_label).item()
        correct_pupilY += torch.sum(pupilY_output == pupilY_label).item()
        correct_pupilR += torch.sum(pupilR_outputs == pupilR_label).item()
        correct_irisX += torch.sum(irisX_output == irisX_label).item()
        correct_irisY += torch.sum(irisY_output == irisY_label).item()
        correct_irisR += torch.sum(irisR_output == irisR_label).item()

    print(f"\nEpoch [{epoch + 1}/{num_epochs}] | Loss Lens: {total_loss_lens / len(train_loaded):.2f} | "
            f"Loss Gender: {total_loss_gender / len(train_loaded):.2f} | "
            f"Loss B/W: {total_loss_black_white / len(train_loaded):.2f} | "
            f"loss_pupilY: {total_loss_pupilY/(len(train_loaded)):.2f} | "
            f"loss_irisX: {total_loss_irisX/(len(train_loaded)):.2f} | "
            f"loss_irisY: {total_loss_irisY/(len(train_loaded)):.2f} | "
            f"loss_pupilR: {total_loss_pupilR/(len(train_loaded)):.2f} | "
            f"loss_irisR: {total_loss_irisR/(len(train_loaded)):.2f}")

    print(f"Gender: {correct_lens * 100 /(len(train_loaded))} |"
          f"B/W: {correct_gender * 100 /(len(train_loaded))} |"
          f"B/W: {correct_black_white * 100 /(len(train_loaded))} |")


    print(
        f"lens: {correct_lens * 100 / (len(train_loaded))} | "
        f"gender: {correct_gender * 100 / (len(train_loaded))} | "
        f"black_white: {correct_black_white * 100 / (len(train_loaded))} | "
        f"pupilX: {correct_pupilX * 100 / (len(train_loaded))} | "
        f"pupilY: {correct_pupilY * 100 / (len(train_loaded))} | "
        f"pupilR: {correct_pupilR * 100 / (len(train_loaded))} | "
        f"irisX: {correct_irisX * 100 / (len(train_loaded))} | "
        f"irisY: {correct_irisY * 100 / (len(train_loaded))} | "
        f"irisR: {correct_irisR * 100 / (len(train_loaded))}\n"
    )

Epoch [1/20] | Batch [20/50] | 
	Loss Lens: 19.69 | Loss Gender: 0.63 | Loss B/W: 0.20 | loss_pupilX: 32345.00 | loss_pupilY: 18306.88 | loss_irisX: 32325.00 | loss_irisY: 19710.62 | loss_pupilR: 4024.38 | loss_irisR: 12309.38


Epoch [1/20] | Batch [40/50] | 
	Loss Lens: 19.57 | Loss Gender: 0.62 | Loss B/W: 0.20 | loss_pupilX: 32041.88 | loss_pupilY: 18145.94 | loss_irisX: 32058.12 | loss_irisY: 19542.19 | loss_pupilR: 3996.25 | loss_irisR: 12154.69
